In [1]:
ord('牛')

29275

In [5]:
chr(0)

'\x00'

In [9]:
ord('0')

48

In [11]:
ord('s')

115

In [16]:
chr(0)

'\x00'

In [15]:
print(chr(0))

 


In [52]:
repr(chr(0))

"'\\x00'"

In [17]:
"this is a test" + chr(0) + "string"

'this is a test\x00string'

In [18]:
print("this is a test" + chr(0) + "string")

this is a test string


In [19]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'


In [21]:
test_string

'hello! こんにちは!'

In [24]:
list(utf8_encoded)

[104,
 101,
 108,
 108,
 111,
 33,
 32,
 227,
 129,
 147,
 227,
 130,
 147,
 227,
 129,
 171,
 227,
 129,
 161,
 227,
 129,
 175,
 33]

In [23]:
print(type(utf8_encoded))

<class 'bytes'>


In [25]:
print(len(test_string))

13


In [26]:
print(len(utf8_encoded))

23


In [27]:
print(utf8_encoded.decode("utf-8"))

hello! こんにちは!


In [29]:
test_string = "hello! こんにちは!"
utf16_encoded = test_string.encode("utf-16")
print(utf16_encoded)

b'\xff\xfeh\x00e\x00l\x00l\x00o\x00!\x00 \x00S0\x930k0a0o0!\x00'


In [30]:
len(utf16_encoded)

28

In [31]:
test_string = "hello! こんにちは!"
utf32_encoded = test_string.encode("utf-32")
print(utf32_encoded)

b'\xff\xfe\x00\x00h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00!\x00\x00\x00 \x00\x00\x00S0\x00\x00\x930\x00\x00k0\x00\x00a0\x00\x00o0\x00\x00!\x00\x00\x00'


In [32]:
len(utf32_encoded)

56

In [33]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

In [53]:
decode_utf8_bytes_to_str_wrong("hello  こ".encode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: unexpected end of data

In [44]:
"hello  こ".encode("utf-8")

b'hello  \xe3\x81\x93'

In [47]:
[b for b in "hello  こ".encode("utf-8")]

[104, 101, 108, 108, 111, 32, 32, 227, 129, 147]

In [55]:
from inspect import getsource

In [62]:
getsource(str.encode)

TypeError: module, class, method, function, traceback, frame, or code object was expected, got method_descriptor

In [135]:
# b'\xa1\xa1\xa3\xa1'.decode('utf-32')

In [195]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
import regex as re
text = 'Mommy and Daddy came to see their house. They praised them and gave them a treat. It was a lemon cake. It was sour, but they liked it. <|endoftext|>They learned that sharing is caring, and that family is sweet.\n'
special_tokens = [re.escape("<|endoftext|>")]
text = re.split("|".join(special_tokens), text)
# x = re.finditer(PAT, text)

In [196]:
text

['Mommy and Daddy came to see their house. They praised them and gave them a treat. It was a lemon cake. It was sour, but they liked it. ',
 'They learned that sharing is caring, and that family is sweet.\n']

In [192]:
for match in x:
    print(match.group())

In [166]:
# for y in [x.encode("utf-8") for x in re.findall(PAT, "some text that i'll pre-tokenize")]:
#     for z in y:
#         # print(z)
#         print(chr(z))

In [1]:
import regex as re
from typing import List, Dict, Tuple, Any
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial
import logging


In [2]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [3]:
def read_corpus_data(file_path: str) -> List[str]:
    "Read corpus data"
    with open(file_path, 'r') as f:
        text_corpus = f.readlines()
    logger.info("Loaded data from: %s", file_path)
    return text_corpus

def initialize_vocab(special_tokens: List[str] = None) -> Dict[bytes, int]:
    "Initialize vocab with initial 256 bytes and special tokens"
    vocab = {chr(i).encode('utf-8'):i for i in range(256)}
    if special_tokens:
        i = len(vocab)
        for special_token in special_tokens:
            vocab[special_token] = i
            i+=1
    logger.info("Initialized vocab with size %i and %s special tokens", len(vocab), ",".join(special_tokens))
    return vocab

def get_byte_sequences(sentence: str, regexp_pattern: str, special_tokens: List[str]) -> List[Tuple[bytes]]:
    "Convert sentence to byte sequences"
    if special_tokens:
        filtered_sentences = re.split("|".join(special_tokens), sentence)
    else:
        filtered_sentences = [sentence]
    for filtered_sentence in filtered_sentences:
        split_sentence = re.finditer(regexp_pattern, filtered_sentence)
        byte_seq = []
        for word_match in split_sentence:
            byte = tuple([x.encode("utf-8") for x in word_match.group()])
            byte_seq.append(byte)
    return byte_seq

def apply_multiprocessing_to_get_byte_sequences(text_corpus: List[str], regexp_pattern: str, special_tokens: List[str]) -> List[List[bytes]]:
    "Apply multiplrocessing to pre tokenization step"
    pool = Pool(processes=(cpu_count() - 6))
    output = pool.map(partial(get_byte_sequences, regexp_pattern=regexp_pattern, special_tokens=special_tokens), text_corpus)
    return output

def pretokenize_text_corpus(text_corpus: List[str], special_tokens: List[str] = None) -> Dict[str, int]:
    "Get count of each byte seq in the text corpus"
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    special_tokens = [re.escape(special_token) for special_token in special_tokens] if special_tokens else special_tokens
    vocab_count = {}
    byte_sequences = apply_multiprocessing_to_get_byte_sequences(text_corpus, PAT, special_tokens)
    for byte_seq in tqdm(byte_sequences):
        for byte in byte_seq:
            if byte in vocab_count:
                vocab_count[byte] +=1
            else:
                vocab_count[byte] = 1
    return vocab_count

def bytes_pair_count(byte_seq: Tuple[bytes], count: int, max_pair: Tuple[bytes]=None) -> Tuple[Dict[Tuple[bytes], int], Tuple[bytes]]:
    "Count the occurance of each pair of bytes in the byte sequence"
    if max_pair:
        byte_seq = list(byte_seq)
        i=0
        while(i<len(byte_seq)-1):
            if max_pair[0]==byte_seq[i] and max_pair[1]==byte_seq[i+1]:
                byte_seq[i] = byte_seq[i]+byte_seq[i+1]
                byte_seq.pop(i+1)
            i+=1
        byte_seq = tuple(byte_seq)
    pair_count = {}
    for i in range(len(byte_seq)-1):
        pair = (byte_seq[i], byte_seq[i+1])
        if pair in pair_count:
            pair_count[pair] += count
        else:
            pair_count[pair] = count
    return pair_count, byte_seq

def merge_dicts(dict1: Dict[bytes, int], dict2: Dict[bytes, int]) -> Dict[bytes, int]:
    "Merge two dicts by adding values of common dict"
    merged_dict = {i: dict1.get(i, 0) + dict2.get(i, 0) for i in set(dict1).union(dict2)}
    return merged_dict

def corpus_pair_count(text_corpus_count: Dict[Tuple[bytes], int], max_pair: Tuple[bytes]=None) -> Tuple[Dict[Tuple[bytes], int], Dict[Tuple[bytes], int]]:
    "Count the occurance of bytes pair in the text corpus"
    pair_counts = {}
    changed_pairs = []
    for i, byte_seq in tqdm(enumerate(text_corpus_count)):
        pair_count, byte_seq_new = bytes_pair_count(byte_seq, text_corpus_count[byte_seq], max_pair)
        if byte_seq_new != byte_seq:
            changed_pairs.append([byte_seq, byte_seq_new])
        pair_counts = merge_dicts(pair_counts, pair_count)
    for changed_pair in changed_pairs:
        text_corpus_count[changed_pair[1]] = text_corpus_count[changed_pair[0]]
        del text_corpus_count[changed_pair[0]]
    return pair_counts, text_corpus_count

def get_max_count_pair(text_corpus_pair_count: Dict[Tuple[bytes], int]) -> bytes:
    "Return pair having maximum count, break ties by preferring the lexicographically greater pair."
    max_pair_count = max(text_corpus_pair_count.values())
    max_pairs = [keys for keys,values in text_corpus_pair_count.items() if values == max_pair_count]
    return max(max_pairs)

def get_vocab(text_corpus_file_path: str, max_vocab_size=350, special_tokens: List[str]=None) -> Dict[bytes, int]:
    "Train bpe and get the vocab"
    logger.info("Load corpus data")
    text_corpus = read_corpus_data(text_corpus_file_path)
    logger.info("corpus data len: %i", len(text_corpus))
    logger.info("Initialize vocab")
    vocab = initialize_vocab(special_tokens)
    i = len(vocab)
    logger.info("Pretokenize text corpus")
    text_corpus_count = pretokenize_text_corpus(text_corpus, special_tokens)
    logger.info("BPE merges")
    max_pair = None
    while(i<max_vocab_size):
        logger.info("merge %i", i)
        text_corpus_pair_count, text_corpus_count = corpus_pair_count(text_corpus_count, max_pair)
        max_pair = get_max_count_pair(text_corpus_pair_count)
        logger.info("max pair %s", max_pair)
        vocab[max_pair[0]+max_pair[1]] = i
        i+=1
    return vocab


In [4]:
special_tokens = ['<|endoftext|>']
max_vocab_size = 1000
text_corpus_file_path = 'data/TinyStoriesV2-GPT4-train.txt'


In [ ]:
vocab = get_vocab(text_corpus_file_path, max_vocab_size, special_tokens)

2025-05-16 01:18:54,029 - INFO - Load corpus data
2025-05-16 01:21:44,156 - INFO - Loaded data from: data/TinyStoriesV2-GPT4-train.txt
2025-05-16 01:21:44,158 - INFO - corpus data len: 15600057
2025-05-16 01:21:44,160 - INFO - Initialize vocab
2025-05-16 01:21:44,161 - INFO - Initialized vocab with size 257 and <|endoftext|> special tokens
2025-05-16 01:21:44,162 - INFO - Pretokenize text corpus


In [463]:
vocab

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [435]:
def apply_multiprocessing(text_corpus: List[str], regexp_pattern: str, special_tokens: List[str]):
    pool = Pool(processes=(cpu_count() - 1))
    results = []
    for sentence in text_corpus:
        result = pool.apply_async(get_byte_sequences, args=(sentence, regexp_pattern, special_tokens))
        results.append(result)
    pool.close()
    pool.join()
    output = []
    for r in results:
        output.extend(r.get())
    return output

In [436]:
special_tokens = ['<|endoftext|>']
max_vocab_size = 260
text_corpus_file_path = 'data/TinyStoriesV2-GPT4-valid.txt'


In [437]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
special_tokens = [re.escape(special_token) for special_token in special_tokens] if special_tokens else special_tokens

In [438]:
text_corpus = read_corpus_data(text_corpus_file_path)


In [439]:
len(text_corpus)

157832

In [440]:
text_corpus = text_corpus[:50]

In [441]:
output = apply_multiprocessing(text_corpus, PAT, special_tokens)

In [442]:
text_corpus

['u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '<|endoftext|>\n',
 'Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\n',
 'Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\n',
 'Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\n',
 'They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called

In [445]:
output

[(b'u',),
 (b' ', b'd', b'o', b'n'),
 (b"'", b't'),
 (b' ', b'h', b'a', b'v', b'e'),
 (b' ', b't', b'o'),
 (b' ', b'b', b'e'),
 (b' ', b's', b'c', b'a', b'r', b'e', b'd'),
 (b' ', b'o', b'f'),
 (b' ', b't', b'h', b'e'),
 (b' ', b'l', b'o', b'u', b'd'),
 (b' ', b'd', b'o', b'g'),
 (b',',),
 (b' ', b'I'),
 (b"'", b'l', b'l'),
 (b' ', b'p', b'r', b'o', b't', b'e', b'c', b't'),
 (b' ', b'y', b'o', b'u'),
 (b'"', b'.'),
 (b' ', b'T', b'h', b'e'),
 (b' ', b'm', b'o', b'l', b'e'),
 (b' ', b'f', b'e', b'l', b't'),
 (b' ', b's', b'o'),
 (b' ', b's', b'a', b'f', b'e'),
 (b' ', b'w', b'i', b't', b'h'),
 (b' ', b't', b'h', b'e'),
 (b' ', b'l', b'i', b't', b't', b'l', b'e'),
 (b' ', b'g', b'i', b'r', b'l'),
 (b'.',),
 (b' ', b'S', b'h', b'e'),
 (b' ', b'w', b'a', b's'),
 (b' ', b'v', b'e', b'r', b'y'),
 (b' ', b'k', b'i', b'n', b'd'),
 (b' ', b'a', b'n', b'd'),
 (b' ', b't', b'h', b'e'),
 (b' ', b'm', b'o', b'l', b'e'),
 (b' ', b's', b'o', b'o', b'n'),
 (b' ', b'c', b'a', b'm', b'e'),
 (b' ', b't',

In [399]:
vocab = get_vocab(text_corpus_file_path, max_vocab_size, special_tokens)

257
mp None


13105it [00:02, 4649.06it/s]


(b' ', b't')
258
mp (b' ', b't')


13105it [00:02, 4578.41it/s]


(b'h', b'e')
259
mp (b'h', b'e')


13105it [00:02, 4393.11it/s]

(b' ', b'a')


In [400]:
vocab

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [395]:
re.split("|".join(special_tokens), 'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n')

TypeError: can only join an iterable

In [394]:
text_corpus[0]

'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n'

In [328]:
def read_corpus_data(file_path: str) -> List[str]:
    with open(file_path, 'r') as f:
        text_corpus = f.readlines()
    return text_corpus

In [329]:
text_corpus_file_path = 'data/TinyStoriesV2-GPT4-valid.txt'
text_corpus = read_corpus_data(text_corpus_file_path)

In [209]:
text_corpus = text_corpus[:100]

In [259]:
# text_corpus

In [334]:
def initialize_vocab(special_tokens: List[str] = None) -> Dict[bytes, int]:
    vocab = {chr(i).encode('utf-8'):i for i in range(256)}
    if special_tokens:
        i = len(vocab)
        for special_token in special_tokens:
            vocab[special_token] = i
            i+=1
    return vocab


In [335]:
special_tokens = ['<|endoftext|>']
vocab = initialize_vocab(special_tokens)

In [343]:
def pretokenize_text_corpus(text_corpus: List[str], special_tokens: List[str] = None) -> Dict[str, int]:
    "Get count of each byte seq in the text corpus"
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    special_tokens = [re.escape(special_token) for special_token in special_tokens]
    vocab_count = {}
    for sentence in text_corpus:
        filtered_sentences = re.split("|".join(special_tokens), sentence)
        for filtered_sentence in filtered_sentences:
            split_sentence = re.finditer(PAT, filtered_sentence)
            for word_match in split_sentence:
                word = tuple([x.encode("utf-8") for x in word_match.group()])# if word_match.group != special_token else word_match.group
                if word in vocab_count:
                    vocab_count[word] +=1
                else:
                    vocab_count[word] = 1
    return vocab_count

In [ ]:
text_corpus_count = pretokenize_text_corpus(text_corpus, special_tokens)

In [255]:
len(text_corpus_count)

658

In [310]:
text_corpus_count

{(b'u',): 1,
 (b' ', b'd', b'o', b'n'): 3,
 (b"'", b't'): 6,
 (b' ', b'h', b'a', b'v', b'e'): 13,
 (b' ', b't', b'o'): 90,
 (b' ', b'b', b'e'): 9,
 (b' ', b's', b'c', b'a', b'r', b'e', b'd'): 6,
 (b' ', b'o', b'f'): 17,
 (b' ', b't', b'h', b'e'): 126,
 (b' ', b'l', b'o', b'u', b'd'): 3,
 (b' ', b'd', b'o', b'g'): 3,
 (b',',): 161,
 (b' ', b'I'): 11,
 (b"'", b'l', b'l'): 2,
 (b' ', b'p', b'r', b'o', b't', b'e', b'c', b't'): 1,
 (b' ', b'y', b'o', b'u'): 28,
 (b'"', b'.'): 1,
 (b' ', b'T', b'h', b'e'): 15,
 (b' ', b'm', b'o', b'l', b'e'): 3,
 (b' ', b'f', b'e', b'l', b't'): 11,
 (b' ', b's', b'o'): 9,
 (b' ', b's', b'a', b'f', b'e'): 5,
 (b' ', b'w', b'i', b't', b'h'): 17,
 (b' ', b'l', b'i', b't', b't', b'l', b'e'): 13,
 (b' ', b'g', b'i', b'r', b'l'): 10,
 (b'.',): 279,
 (b' ', b'S', b'h', b'e'): 30,
 (b' ', b'w', b'a', b's'): 59,
 (b' ', b'v', b'e', b'r', b'y'): 15,
 (b' ', b'k', b'i', b'n', b'd'): 4,
 (b' ', b'a', b'n', b'd'): 130,
 (b' ', b's', b'o', b'o', b'n'): 1,
 (b' ', b'c', b'

In [231]:
max_vocab = 350

In [367]:
def bytes_pair_count(byte_seq: Tuple[bytes], count: int, max_pair: Tuple[bytes]=None) -> Tuple[Dict[Tuple[bytes], int], Tuple[bytes]]:
    "Count the occurance of each pair of bytes in the byte sequence"
    if max_pair:
        byte_seq = list(byte_seq)
        i=0
        while(i<len(byte_seq)-1):
            if max_pair[0]==byte_seq[i] and max_pair[1]==byte_seq[i+1]:
                byte_seq[i] = byte_seq[i]+byte_seq[i+1]
                byte_seq.pop(i+1)
            i+=1
        byte_seq = tuple(byte_seq)
    pair_count = {}
    for i in range(len(byte_seq)-1):
        pair = (byte_seq[i], byte_seq[i+1])
        if pair in pair_count:
            pair_count[pair] += count
        else:
            pair_count[pair] = count
    return pair_count, byte_seq

In [368]:
def merge_dicts(dict1: Dict[bytes, int], dict2: Dict[bytes, int]) -> Dict[bytes, int]:
    "Merge two dicts by adding values of common dict"
    merged_dict = {i: dict1.get(i, 0) + dict2.get(i, 0) for i in set(dict1).union(dict2)}
    return merged_dict

In [379]:
def corpus_pair_count(text_corpus_count: Dict[Tuple[bytes], int], max_pair: Tuple[bytes]=None) -> Tuple[Dict[Tuple[bytes], int], Dict[Tuple[bytes], int]]:
    "Count the occurance of bytes pair in the text corpus"
    pair_counts = {}
    changed_pairs = []
    for i, byte_seq in tqdm(enumerate(text_corpus_count)):
        pair_count, byte_seq_new = bytes_pair_count(byte_seq, text_corpus_count[byte_seq], max_pair)
        if byte_seq_new != byte_seq:
            changed_pairs.append([byte_seq, byte_seq_new])
        pair_counts = merge_dicts(pair_counts, pair_count)
    for changed_pair in changed_pairs:
        text_corpus_count[changed_pair[1]] = text_corpus_count[changed_pair[0]]
        del text_corpus_count[changed_pair[0]]
    return pair_counts, text_corpus_count

In [380]:
def get_max_count_pair(text_corpus_pair_count: Dict[Tuple[bytes], int]) -> bytes:
    "Return pair having maximum count, break ties by preferring the lexicographically greater pair."
    max_pair_count = max(text_corpus_pair_count.values())
    max_pairs = [keys for keys,values in text_corpus_pair_count.items() if values == max_pair_count]
    return max(max_pairs)

In [381]:
def get_vocab(text_corpus_file_path: str, max_vocab_size=350, special_tokens: List[str]=None):
    text_corpus = read_corpus_data(text_corpus_file_path)
    vocab = initialize_vocab(special_tokens)
    i = len(vocab)
    text_corpus_count = pretokenize_text_corpus(text_corpus, special_tokens)
    max_pair = None
    while(i<max_vocab_size):
        print(i)
        print('mp', max_pair)
        text_corpus_pair_count, text_corpus_count = corpus_pair_count(text_corpus_count, max_pair)
        max_pair = get_max_count_pair(text_corpus_pair_count)
        print(max_pair)
        vocab[max_pair[0]+max_pair[1]] = i
        i+=1
    return vocab

In [389]:
special_tokens = ['<|endoftext|>']
max_vocab_size = 350
text_corpus_file_path = 'data/TinyStoriesV2-GPT4-valid.txt'


In [390]:
vocab = get_vocab(text_corpus_file_path, max_vocab_size, special_tokens)

257
mp None


13105it [00:03, 3651.34it/s]


(b' ', b't')
258
mp (b' ', b't')


13105it [00:03, 3361.07it/s]


(b'h', b'e')
259
mp (b'h', b'e')


...

(b'n', b't')
348
mp (b'n', b't')


13105it [00:08, 1526.61it/s]


(b'c', b'k')
349
mp (b'c', b'k')


13105it [00:08, 1494.95it/s]

(b'ver', b'y')


In [391]:
len(vocab)

350

In [392]:
vocab

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [355]:
vocab

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [314]:
text_corpus_pair_count = corpus_pair_count(text_corpus_count)    

100%|██████████| 658/658 [00:00<00:00, 10120.73it/s]


In [315]:
len(text_corpus_pair_count)

382

In [316]:
max_pair = get_max_count_pair(text_corpus_pair_count)

In [322]:
print(max_pair[0]+max_pair[1])

b'he'


In [ ]:
vocab[max_pair[0]+max_pair[1]] = i 
i+=1

In [324]:
vocab

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [291]:
max(text_corpus_pair_count, key=text_corpus_pair_count.get)

(b'h', b'e')

In [307]:
text_corpus_pair_count[(b'a', b'a')] = 389

In [295]:
max(text_corpus_pair_count, key=text_corpus_pair_count.get)

(b'n', b'a')

In [296]:
max_val = [keys for keys,values in text_corpus_pair_count.items() if values == max(text_corpus_pair_count.values())]

In [297]:
max_val = [(x[0], x[1]) for x in max_val]

In [298]:
max([str(x) for x in max_val])

"(b'n', b'a')"